In [2]:
import phe
from phe import paillier
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

%matplotlib inline

In [3]:
df = pd.read_csv('auto-mpg.csv')
df = df[df.horsepower != '?']
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [4]:
X = df[['cylinders','displacement','horsepower','weight','acceleration','model year','origin']].values.astype(np.float32)
Y = df['mpg'].values.astype(np.float32)
X1 = X[:int(X.shape[0]/2),:]
X2 = X[int(X.shape[0]/2):,:]
Y1 = Y[:int(X.shape[0]/2)]
Y2 = Y[int(X.shape[0]/2):]
X1.shape, Y1.shape, X2.shape, Y2.shape

((196, 7), (196,), (196, 7), (196,))

In [5]:
X1 = np.vstack((X1, np.zeros((X.shape[0] - X1.shape[0],X.shape[1]))))
X2 = np.vstack((np.zeros((X.shape[0] - X2.shape[0],X.shape[1])), X2))
Y1 = np.concatenate((Y1, np.zeros(Y.shape[0] - Y1.shape[0])))
Y2 = np.concatenate((np.zeros(Y.shape[0] - Y2.shape[0]), Y2))
X1T = X1.T
X2T = X2.T

In [6]:
X1.shape, X2.shape, X1.T.shape, X2.T.shape, Y1.shape, Y2.shape

((392, 7), (392, 7), (7, 392), (7, 392), (392,), (392,))

In [7]:
#np.dot(X1T, X1) + np.dot(X1T, X2) + np.dot(X2T, X1) + np.dot(X2T, X2)

In [8]:
#np.dot(X.T, X)

In [9]:
#np.dot(X1T, Y1) + np.dot(X1T, Y2) + np.dot(X2T, Y1) + np.dot(X2T, Y2)

In [10]:
#np.dot(X.T, Y)
def calculate_number_inverse(c):
    x = random.choice(np.linspace(0, 2/c, 11)[1:])
    for i in range(15):
        x = x * (2 - c * x)
    return x

calculate_number_inverse(200.5), 1/ 200.5, calculate_number_inverse(2175848264.6200633+1608062424.53014), 1 / (2175848264.6200633+1608062424.53014)

(0.004987531172069825,
 0.004987531172069825,
 2.642768506316362e-10,
 2.642768506316362e-10)

In [22]:
no_of_users = 2
part_per_user = int(df.shape[0] / no_of_users)
user_shares = []
start = 0
end = part_per_user
for i in range(no_of_users):
    print(start, end)
    if i == no_of_users-1:
        end = df.shape[0]
        user_shares.append((start, end, df.iloc[start:,:]))
        print(end)
        continue
    user_shares.append((start,end, df.iloc[start:end, :]))
    start = end
    end += part_per_user
user_shares[0][2].shape, user_shares[1][2].shape,

0 196
196 392
392


((196, 9), (196, 9))

In [118]:
class User(object):
    def __init__(self,df, X_columns, Y_column,total_size,start,end, pub):
        X = df[X_columns].values.astype(np.float32)
        Y = df[Y_column].values.astype(np.float32)
        self.pub_key = pub
        
        print(X.dtype,Y.dtype)
        if start == 0:
            self.X = np.vstack((X, np.zeros((total_size-end, X.shape[1]))))
            self.Y = np.concatenate((Y, np.zeros(total_size - end)))
        elif end == total_size:
            self.X = np.vstack((np.zeros((start, X.shape[1])), X))
            self.Y = np.concatenate((np.zeros(start), Y))
        else:
            self.X = np.vstack((np.zeros((start-1,X.shape[1])), X, np.zeros((total_size-end,X.shape[1]))))
            self.Y = np.concatenate((np.zeros((start-1,)), Y, np.zeros((total_size-end,))))
        print(self.X.shape, self.Y.shape)
        
        M = pow(2,64)
        #self.X = self.X * M
        #self.Y = self.Y * M
        self.XT = self.X.T
        self.X_enc = self.encrypt(self.X)
        self.X_enc_T = self.X_enc.T
        self.Y_enc = self.encrypt(self.Y)
        #self.encrypt()
        #print(X)
        #print(Y)
    def encrypt(self,X):
        X_new = np.empty(X.shape,dtype=np.object)
        if len(X.shape) == 2:
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    X_new[i][j] = self.pub_key.encrypt(X[i][j],precision=1e-15)
        else:
            for i in range(X.shape[0]):
                X_new[i] = self.pub_key.encrypt(X[i], precision=1e-15)
                
        return X_new
    
    def decrypt(self,X,priv_key):  
        X_new = np.empty(X.shape,dtype=np.float32)
        if len(X.shape) == 2:
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    X_new[i][j] = priv_key.decrypt(X[i][j])
        else:
            for i in range(X.shape[0]):
                X_new[i] = priv_key.decrypt(X[i])
        return X_new
    
    def sendEncryptedXYtoGetParts(self):
        pass
    
    @staticmethod
    def calculate_number_inverse(c):
        x = random.choice(np.linspace(0, 2/c, 11)[1:])
        for i in range(10):
            x *= (2 - c * x)
        return x
    
    @staticmethod
    def calculate_matrix_inverse(A):
        #A = self.decrypt_matrix(A)
        c = np.trace(A)
        I = np.eye(A.shape[0])
        #c_inv = calculate_number_inverse(c)
        c_inv = User.calculate_number_inverse(c)
        print(c, c_inv)
        A0 = c_inv * I
        M0 = c_inv * A
        A = (2 * A0) - (A0 * M0)
        M = (2 * M0) - (M0 ** 2)
        for i in range(1,10):
            A = (2 * A) - (A * M)
            M = (2 * M) - (M ** 2)
            if np.allclose(M, np.eye(A.shape[0]), rtol = 0.001, atol = 0.001, equal_nan=True):
                break
            print(M)
        print(A)
    
    @staticmethod
    def calculate_matrix_inverse2(A):
        c = np.trace(A)
        I = np.eye(A.shape[0])
        c_inv = User.calculate_number_inverse(c)
        X0 = c_inv * I
        X = X0 * (2 - A * X0)
        print(X)
        for _ in range(5):
            X = X * (2 - A * X)
            
        print(X)

In [119]:
%%time
pub, priv = paillier.generate_paillier_keypair(n_length = 1028)
user1 = User(user_shares[0][2], 
             ['cylinders','displacement','horsepower','weight','acceleration','model year','origin'],
             'mpg',
             Y.shape[0],
             user_shares[0][0],
             user_shares[0][1],
             pub
            )
user2 = User(user_shares[1][2], 
             ['cylinders','displacement','horsepower','weight','acceleration','model year','origin'],
             'mpg',
             Y.shape[0],
             user_shares[1][0],
             user_shares[1][1],
             pub
            )

float32 float32
(392, 7) (392,)
float32 float32
(392, 7) (392,)
CPU times: user 3min 12s, sys: 5.43 s, total: 3min 18s
Wall time: 3min 18s


In [120]:
%%time

## Calculate XT*X shares
A1 = np.dot(user1.XT,user1.X)
A2 = np.dot(user2.XT,user2.X)

A_part1 = np.empty((user1.X.shape[1],user1.X.shape[1]),dtype=np.object)
A_part2 = np.empty((user2.X.shape[1],user2.X.shape[1]),dtype=np.float32)
temp = np.dot(user1.X_enc_T, user2.X) + np.dot(user2.XT, user1.X_enc)

for i in range(temp.shape[0]):
    for j in range(temp.shape[1]):
        r = random.randint(0,1000)
        A_part1[i][j] = temp[i][j] - r
        A_part2[i][j] = r

A_part1 = user1.decrypt(A_part1,priv)
A1 = A1 + A_part1
A2 = A2 + A_part2

A1_enc = user1.encrypt(A1)

CPU times: user 34.6 s, sys: 92.3 ms, total: 34.7 s
Wall time: 34.7 s


In [221]:
%%time

## Calculate XT*Y shares
B1 = np.dot(user1.XT, user1.Y)
B2 = np.dot(user2.XT, user2.Y)

B_part1 = np.empty(user1.XT.shape[0],dtype=np.object)
B_part2 = np.empty(user1.XT.shape[0],dtype=np.float32)
temp = np.dot(user1.X_enc_T, user2.Y) + np.dot(user2.XT, user1.Y_enc)

for i in range(temp.shape[0]):
    r = B2[i] / 2
    B_part1[i] = temp[i] - r
    B_part2[i] = r
    
B_part1 = user1.decrypt(B_part1,priv)
B1 = B1 + B_part1
B2 = B2 + B_part2

CPU times: user 4.84 s, sys: 12 ms, total: 4.86 s
Wall time: 4.87 s


In [222]:
B1 + B2

array([4.62435000e+04, 1.52968590e+06, 8.68718800e+05, 2.52090614e+07,
       1.46401390e+05, 7.04839600e+05, 1.58790000e+04])

In [220]:
np.dot(X.T,Y)

array([4.6243512e+04, 1.5296858e+06, 8.6871875e+05, 2.5209068e+07,
       1.4640141e+05, 7.0483962e+05, 1.5879000e+04], dtype=float32)

In [193]:
A1+A2

array([[1.28750000e+04, 4.83375000e+05, 2.45728000e+05, 6.89538400e+06,
        3.24075000e+04, 1.62127000e+05, 3.07600000e+03],
       [4.83375000e+05, 1.90976342e+07, 9.37464700e+06, 2.59345480e+08,
        1.12301190e+06, 5.73462100e+06, 9.98925000e+04],
       [2.45728000e+05, 9.37464700e+06, 4.85752400e+06, 1.32989885e+08,
        6.07832300e+05, 3.08843300e+06, 5.90440000e+04],
       [6.89538400e+06, 2.59345480e+08, 1.32989885e+08, 3.75757549e+09,
        1.77581036e+07, 8.83061810e+07, 1.68364300e+06],
       [3.24075000e+04, 1.12301190e+06, 6.07832300e+05, 1.77581036e+07,
        9.76569002e+04, 4.64036500e+05, 9.78940000e+03],
       [1.62127000e+05, 5.73462100e+06, 3.08843300e+06, 8.83061810e+07,
        4.64036500e+05, 2.26828200e+06, 4.71660000e+04],
       [3.07600000e+03, 9.98925000e+04, 5.90440000e+04, 1.68364300e+06,
        9.78940000e+03, 4.71660000e+04, 1.22800000e+03]])

In [125]:
np.dot(X.T,X)

array([[1.2875000e+04, 4.8337500e+05, 2.4572800e+05, 6.8953840e+06,
        3.2407502e+04, 1.6212700e+05, 3.0760000e+03],
       [4.8337500e+05, 1.9097636e+07, 9.3746470e+06, 2.5934547e+08,
        1.1230119e+06, 5.7346210e+06, 9.9892500e+04],
       [2.4572800e+05, 9.3746470e+06, 4.8575240e+06, 1.3298987e+08,
        6.0783225e+05, 3.0884330e+06, 5.9044000e+04],
       [6.8953840e+06, 2.5934547e+08, 1.3298987e+08, 3.7575764e+09,
        1.7758104e+07, 8.8306184e+07, 1.6836430e+06],
       [3.2407502e+04, 1.1230119e+06, 6.0783225e+05, 1.7758104e+07,
        9.7656891e+04, 4.6403650e+05, 9.7893994e+03],
       [1.6212700e+05, 5.7346210e+06, 3.0884330e+06, 8.8306184e+07,
        4.6403650e+05, 2.2682820e+06, 4.7166000e+04],
       [3.0760000e+03, 9.9892500e+04, 5.9044000e+04, 1.6836430e+06,
        9.7893994e+03, 4.7166000e+04, 1.2280000e+03]], dtype=float32)

In [126]:
ca1, ca2 = np.trace(A1), np.trace(A2)
ca1,ca2, 1 / (ca1+ca2), 1 / np.trace(np.dot(X.T,X)), ca1+ca2, np.trace(np.dot(X.T,X)), (ca1+ca2) == np.trace(np.dot(X.T,X))

(2175849137.6200633,
 1608061551.53014,
 2.642768506316362e-10,
 2.642767993080802e-10,
 3783910689.150203,
 3783911400.0,
 False)

In [147]:
%%time
xa1 = random.choice(np.linspace(0, 2/ca1, 11)[1:])
xa2 = random.choice(np.linspace(0, 2/ca2, 11)[1:])
xa1,xa2 = 1e-12, 1e-12
ca1_enc = pub.encrypt(ca1, precision=1e-15)
xa1_enc = pub.encrypt(xa1, precision=1e-15)
xa1_sq_enc = pub.encrypt(xa1*xa1, precision=1e-15)
ca1xa1_enc = pub.encrypt(ca1*xa1, precision=1e-15)
#print(xa1,xa2)
for i in range(10):
    xa1 = 2*xa1 - (ca1*xa1*xa1)
    xa2 = 2*xa2 - (ca2*xa2*xa2)
    temp = (2*ca1xa1_enc*xa2) + (ca1_enc*xa2*xa2) + (ca2*xa1_sq_enc) + (2*ca2*xa1_enc*xa2)
    r = xa2 / 4
    a1 = temp - r
    a2 = r
    a1 = priv.decrypt(a1)
    #print(a1)
    xa1 -= a1
    xa2 -= a2
    #print(xa1, xa2)
    xa1_enc = pub.encrypt(xa1, precision=1e-15)
    xa1_sq_enc = pub.encrypt(xa1*xa1, precision=1e-15)
    ca1xa1_enc = pub.encrypt(ca1*xa1, precision=1e-15)

print(xa1+xa2)
cinv_a1, cinv_a2 = xa1,xa2

2.9469982052434803e-10
CPU times: user 1.37 s, sys: 19.9 ms, total: 1.39 s
Wall time: 1.39 s


In [148]:
#xa1 = random.choice(np.linspace(0, 2/ca1, 11)[1:])
#xa2 = random.choice(np.linspace(0, 2/ca2, 11)[1:])
#xa1,xa2 = 1e-12, 1e-12
#print(xa1,xa2)
#for i in range(50):
#    xa1 = 2*xa1 - (ca1*xa1*xa1)
#    xa2 = 2*xa2 - (ca2*xa2*xa2)
#    temp = (2*ca1*xa1*xa2) + (ca1*xa2*xa2) + (ca2*xa1*xa1) + (2*ca2*xa1*xa2)
#    print(temp)
#    r = xa2 / 2 #random.choice(np.linspace(0, 2/ca2,11)[1:])
#    a1 = temp - r
#    a2 = r
#    print(a1,a2)
#    xa1 -= a1
#    xa2 -= a2
#    print(xa1, xa2)

#print(xa1+xa2)
#cinv_a1, cinv_a2 = xa1,xa2

In [149]:
#cinv_a1, cinv_a2 = 1 / (2*(ca1+ca2)), 1 / (2*(ca1+ca2))
cinv_a1, cinv_a2, priv.decrypt(pub.encrypt(cinv_a1,precision=1e-15))

(2.41931715157794e-10, 5.2768105366554025e-11, 2.419318079205368e-10)

In [170]:
%%time

X01 = cinv_a1 * np.eye(user1.X.shape[1])
X02 = cinv_a2 * np.eye(user2.X.shape[1])
M01 = cinv_a1 * A1
M02 = cinv_a2 * A2
cinv_a1_enc = pub.encrypt(cinv_a1,precision=1e-15)
temp = np.empty(X01.shape,dtype=np.object)
for i in range(temp.shape[0]):
    for j in range(temp.shape[1]):
        temp[i][j] = cinv_a1_enc * A2[i][j]

temp += (A1_enc * cinv_a2)
M0_part1 = np.empty(A1.shape,dtype=np.object)
M0_part2 = np.empty(A2.shape,dtype=np.float32)

for i in range(temp.shape[0]):
    for j in range(temp.shape[1]):
        r = M02[i][j] / 2
        M0_part1[i][j] = temp[i][j] - r
        M0_part2[i][j] = r

M0_part1 = user1.decrypt(M0_part1, priv)
M01 += M0_part1
M02 += M0_part2
#print(X01+X02, M01+M02)
X1 = 2*X01 - np.dot(X01, M01)
X2 = 2*X02 - np.dot(X02, M02)
M1 = 2*M01 - np.dot(M01, M01)
M2 = 2*M02 - np.dot(M02, M02)

X01_enc = user1.encrypt(X01)
M01_enc = user1.encrypt(M01)
temp = np.dot(X01_enc, M02) + np.dot(X02, M01_enc)
X1_part = np.empty(X1.shape,dtype=np.object)
X2_part = np.empty(X2.shape,dtype=np.float32)

for i in range(temp.shape[0]):
    for j in range(temp.shape[1]):
        r = X2[i][j] / 2
        X1_part[i][j] = temp[i][j] - r
        X2_part[i][j] = r

X1 -= user1.decrypt(X1_part, priv)
X2 -= X2_part

temp = np.dot(M01_enc, M02) * 2
M1_part = np.empty(M1.shape,dtype=np.object)
M2_part = np.empty(M2.shape,dtype=np.float32)

for i in range(temp.shape[0]):
    for j in range(temp.shape[1]):
        r = M2[i][j] / 2
        M1_part[i][j] = temp[i][j] - r
        M2_part[i][j] = r

M1 -= user1.decrypt(M1_part, priv)
M2 -= M2_part
#print(X1+X2, M1+M2)

for i in range(30):
    X1_enc = user1.encrypt(X1)
    M1_enc = user1.encrypt(M1)
    X1 = 2*X1 - np.dot(X1, M1)
    X2 = 2*X2 - np.dot(X2, M2)
    temp = np.dot(X1_enc, M2) + np.dot(X2, M1_enc)
    X1_part = np.empty(X1.shape,dtype=np.object)
    X2_part = np.empty(X2.shape,dtype=np.float32)

    for i in range(temp.shape[0]):
        for j in range(temp.shape[1]):
            r = X2[i][j] / 2
            X1_part[i][j] = temp[i][j] - r
            X2_part[i][j] = r

    X1 -= user1.decrypt(X1_part, priv)
    X2 -= X2_part
    M1 = 2*M1 - np.dot(M1, M1)
    M2 = 2*M2 - np.dot(M2, M2)
    temp = np.dot(M1_enc, M2) * 2
    M1_part = np.empty(M1.shape,dtype=np.object)
    M2_part = np.empty(M2.shape,dtype=np.float32)

    for i in range(temp.shape[0]):
        for j in range(temp.shape[1]):
            r = M2[i][j] / 2
            M1_part[i][j] = temp[i][j] - r
            M2_part[i][j] = r

    M1 -= user1.decrypt(M1_part, priv)
    M2 -= M2_part
    
A1_inv, A2_inv = X1, X2

CPU times: user 3min 33s, sys: 2.83 s, total: 3min 36s
Wall time: 3min 36s


In [198]:
A1_inv+A2_inv, np.linalg.inv(np.dot(X.T,X))

(array([[ 9.14176246e-03, -1.42853291e-04,  1.76228804e-05,
         -1.75101134e-06, -7.87048444e-05, -2.09820975e-04,
         -9.64202094e-04],
        [-1.42829330e-04,  5.07422096e-06, -2.45013762e-06,
         -1.81025536e-07,  8.94589711e-06,  4.47894663e-06,
          6.13591682e-05],
        [ 1.76328459e-05, -2.44993218e-06,  1.37630256e-05,
         -2.95863157e-07,  5.19226754e-05, -1.05792460e-05,
         -9.87120105e-05],
        [-1.82546703e-06, -1.76279953e-07, -3.07643352e-07,
          3.86327925e-08, -2.36597086e-06, -6.06041231e-09,
          1.87400309e-06],
        [-7.87198311e-05,  8.95072965e-06,  5.19282918e-05,
         -2.24041958e-06,  7.11436924e-04, -1.39360143e-04,
         -1.47758965e-04],
        [-2.09814749e-04,  4.47822028e-06, -1.05806516e-05,
         -4.67745230e-08, -1.39364008e-04,  5.06872598e-05,
         -1.39319502e-04],
        [-9.64259989e-04,  6.13706388e-05, -9.87229484e-05,
          2.02355220e-06, -1.47807570e-04, -1.39311340e-04

In [227]:
%%time

beta1 = np.dot(A1_inv, B1)
beta2 = np.dot(A2_inv, B2)
#print(beta1.shape,beta2.shape)
A1_inv_enc = user1.encrypt(A1_inv)
B1_enc = user1.encrypt(B1)
temp = np.dot(A1_inv_enc, B2) + np.dot(A2_inv, B1_enc)
#print(temp.shape)
beta_part1 = np.empty(beta1.shape,dtype=np.object)
beta_part2 = np.empty(beta2.shape,dtype=np.float32)

for i in range(temp.shape[0]):
    r = beta2[i] / 2
    beta_part1[i] = temp[i] - r
    beta_part2[i] = r
    
beta1 += user1.decrypt(beta_part1,priv)
beta2 += beta_part2

CPU times: user 2.09 s, sys: 71.8 ms, total: 2.16 s
Wall time: 2.16 s


In [228]:
beta1+beta2, np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,Y))

(array([ 0.67120674, -0.09397539,  0.14297666,  0.03167619,  2.26627548,
        -0.11200166,  4.18490103]),
 array([-0.70539665,  0.02186924, -0.03956068, -0.00603187, -0.07965064,
         0.58360624,  1.3099518 ], dtype=float32))

In [159]:
def calculate_matrix_inverse(A):
    c = np.trace(A)
    I = np.eye(A.shape[0])
    #c_inv = calculate_number_inverse(c)
    c_inv = 1 / c
    print(c, c_inv)
    X0 = c_inv * I
    M0 = c_inv * A
    print(X0, M0)
    X = (2 * X0) - np.dot(X0, M0)
    M = (2 * M0) - np.dot(M0, M0)
    print(X, M)
    for i in range(1,50):
        X = (2 * X) - np.dot(X, M)
        M = (2 * M) - np.dot(M, M)
        #if np.allclose(M, np.eye(A.shape[0]), rtol = 0.001, atol = 0.001, equal_nan=True):
        #    break
    print(M, X)

In [160]:
INV = np.random.randint(500,50000,size=9).reshape(3,3)
INV = np.array([[4.5,7.8],[2.3,6.1]])

In [161]:
np.linalg.inv(INV)

array([[ 0.64143007, -0.82018927],
       [-0.24185068,  0.47318612]])

In [162]:
calculate_matrix_inverse(INV)

10.6 0.09433962264150944
[[0.09433962 0.        ]
 [0.         0.09433962]] [[0.4245283  0.73584906]
 [0.21698113 0.5754717 ]]
[[ 0.14862941 -0.06941972]
 [-0.02046992  0.13438946]] [[0.50916696 0.73584906]
 [0.21698113 0.66011036]]
[[1. 0.]
 [0. 1.]] [[ 0.64143007 -0.82018927]
 [-0.24185068  0.47318612]]


In [163]:
np.linalg.inv(A1+A2)

array([[ 9.14241146e-03, -1.42848138e-04,  1.77914008e-05,
        -1.80735848e-06, -7.75064333e-05, -2.10298271e-04,
        -9.62914694e-04],
       [-1.42848138e-04,  5.07443810e-06, -2.45024184e-06,
        -1.76435712e-07,  8.94098435e-06,  4.48111092e-06,
         6.13570421e-05],
       [ 1.77914008e-05, -2.45024184e-06,  1.38078894e-05,
        -3.03436484e-07,  5.22451696e-05, -1.07069538e-05,
        -9.83750430e-05],
       [-1.80735848e-06, -1.76435712e-07, -3.03436484e-07,
         3.71128039e-08, -2.33535849e-06, -1.83463299e-08,
         1.90758288e-06],
       [-7.75064333e-05,  8.94098435e-06,  5.22451696e-05,
        -2.33535849e-06,  7.13720854e-04, -1.40271797e-04,
        -1.45293693e-04],
       [-2.10298271e-04,  4.48111092e-06, -1.07069538e-05,
        -1.83463299e-08, -1.40271797e-04,  5.10494778e-05,
        -1.40313337e-04],
       [-9.62914694e-04,  6.13570421e-05, -9.83750430e-05,
         1.90758288e-06, -1.45293693e-04, -1.40313337e-04,
         6.8973466

In [164]:
calculate_matrix_inverse(A1+A2)

3783910689.150203 2.642768506316362e-10
[[2.64276851e-10 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.64276851e-10 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.64276851e-10 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.64276851e-10
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.64276851e-10 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.64276851e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.64276851e-10]] [[3.40256445e-06 1.27744823e-04 6.49402220e-05 1.82229037e-03
  8.56455204e-06 4.28464130e-05 8.12915593e-07]
 [1.27744823e-04 5.04706263e-03 2.47750218e-03 6.85390067e-02
  2.96786048e-04 

In [25]:
class PaillierSymmetric(object):
    
    class PaillierNumber(object):
        def __init__(self,cipher, nsquare):
            self.cipher = cipher
            self.nsquare = nsquare

        def __mul__(self,c2):
            return pow(self.cipher,c2, self.nsquare) if isinstance(c2,int) else None

        def __add__(self,c2):
            return (self.cipher * c2.cipher) % (self.nsquare)

    def __init__(self,pub,priv):
        
        self.n, self.g = pub.n, pub.g
        self.p, self.q = priv.p, priv.q
        self.nsquare = pub.nsquare
        self.gLambda = PaillierSymmetric.lcm(self.p-1,self.q-1)
        self.r = pub.get_random_lt_n()
        self.l = (pow(self.g, self.gLambda, self.nsquare)-1)//self.n
        self.gMu = PaillierSymmetric.inverse_of(self.l, self.n)
        
        if (PaillierSymmetric.gcd(self.g,self.nsquare)==1):
            print("g is relatively prime to n*n")
        else:
            print("WARNING: g is NOT relatively prime to n*n. Will not work!!!")
        
    @staticmethod
    def gcd(x, y):
        while(y):
            x, y = y, x % y
        return x

    @staticmethod
    def lcm(x, y):
        lcm = (x*y)//PaillierSymmetric.gcd(x,y)
        return lcm
        
    @staticmethod
    def extended_euclidean_algorithm(a, b):
        """
        Returns a three-tuple (gcd, x, y) such that
        a * x + b * y == gcd, where gcd is the greatest
        common divisor of a and b.

        This function implements the extended Euclidean
        algorithm and runs in O(log b) in the worst case.
        """
        s, old_s = 0, 1
        t, old_t = 1, 0
        r, old_r = b, a

        while r != 0:
            quotient = old_r // r
            old_r, r = r, old_r - quotient * r
            old_s, s = s, old_s - quotient * s
            old_t, t = t, old_t - quotient * t

        return old_r, old_s, old_t

    @staticmethod
    def inverse_of(n, p):
        """
        Returns the multiplicative inverse of
        n modulo p.

        This function returns an integer m such that
        (n * m) % p == 1.
        """
        gcd, x, y = PaillierSymmetric.extended_euclidean_algorithm(n, p)
        assert (n * x + p * y) % p == gcd

        if gcd != 1:
            # Either n is 0, or p is not a prime number.
            raise ValueError(
                '{} has no multiplicative inverse '
                'modulo {}'.format(n, p))
        else:
            return x % p
    
    def encrypt(self,m):
        k1 = pow(self.g, m, self.nsquare)
        k2 = pow(self.r, self.n, self.nsquare)
        cipher = (k1 * k2) % self.nsquare
        return self.PaillierNumber(cipher,self.nsquare)
    
    def decrypt(self,cipher):
        l = (pow(cipher, self.gLambda, self.nsquare)-1) // self.n
        mess= (l * self.gMu) % self.n

        return mess

In [26]:
pub, priv = paillier.generate_paillier_keypair(n_length = 16)
pal = PaillierSymmetric(pub,priv)

g is relatively prime to n*n


In [27]:
pn = pal.encrypt(100)
pal.decrypt(pn * 10),pal.decrypt(pn + pal.encrypt(12))

In [46]:
M = pow(2,64)
P = pow(2,128)
m = 3 * M
if m <= (pub.n//2):
    p = m
else:
    p = m - pub.n

print((2*m) < P)

r1 = random.randint(P+1,pub.n-2)
r1 = 150

pub, priv = paillier.generate_paillier_keypair(n_length=1024)
enc_p = pub.encrypt(p)
s = enc_p - r1 
s = priv.decrypt(s)
p1 = (s - pub.n) % P
p2 = r1 % P
enc_p1 = pub.encrypt(p1)
enc_p2 = pub.encrypt(p2)
enc_p1_plus_p2 = enc_p1 + enc_p2
enc_p_hat = enc_p1_plus_p2 - pub.encrypt(p)
m_p_hat = enc_p_hat / M
r2 = 100
enc_m_hat_minus_r = m_p_hat - pub.encrypt(r2)
s = priv.decrypt(enc_m_hat_minus_r)
s1 = ((p1//M) - s) % pub.n
s2 = ((p2//M) - r2) % pub.n

True
